<figure>
    <img src='./logo.png' alt="logo" class="bg-primary" align="right" style="width: 150px; height: 150px;"/>
</figure>

<a id='top'></a>
# NIRSpec IFU Pipeline Processing for extended sources: ERO 2729 - Tarantula Nebula
<hr style="border:3px solid black"  width=80% align="left">


## About this Notebook <a id='about'></a>
<hr style="border:1px solid gray" width=80% align="left">

**Authors**: Peter Zeidler (zeidler@stsci.edu) based on the work by Kayli Glidic (kglidic@stsci.edu), Maria Pena-Guerrero (pena@stsci.edu), Leonardo Ubeda (lubeda@stsci.edu)

**Update On**: 2023-11-28

## Table of Contents

* [1. Introduction](#intro)
* [2. Import Library](#imports)
* [3. Convenience Functions](#func)
* [4. Directory Set-Up](#dir_setup)
* [5. Download the data](#data)
* [6. Products Found In MAST](#mast_products)
    * [6.1 Stage 1 Products Found In MAST](#level1_mast)
    * [6.2 Stage 2 Products Found In MAST](#level2_mast)
    * [6.3 Stage 3 Products Found In MAST](#level3_mast)
* [7. Re-processing the Data](#reprocessing)
    * [7.1 Stage 1 Rerun & Products](#level1_rerun)
    * [7.2 Stage 2 Rerun & Products](#level2_rerun)
    * [7.3 Stage 3 Rerun & Products](#level3_rerun)
        * [7.3.1 New Outlier Detection Algorithm](#outlier_detection_new)
* [Conclusion](#conclusion)

## 1. Introduction <a id='intro'></a>
<hr style="border:1px solid gray">

End-to-end calibration of JWST data is divided into 3 main stages of processing. This notebook explores how to run the JWST calibration pipeline stages 1-3 for NIRSpec IFU spectroscopic data.
   <figure>
       <img src='./nirspec_ifu_extended_source/Tarantula.png' title="Figure 1: Tarantula Nebula" alt="Tarantula" class="bg-primary" align="right" style="width: 400px; height: 350px;"/>
   </figure>

>* **`STAGE 1`** ([calwebb_detector1](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1)): consists of detector-level corrections, performed on a group-by-group basis, followed by ramp fitting.
    * **Input**: Raw exposure (`uncal.fits`) containing original data from all detector readouts (ncols x nrows x ngroups x nintegrations).
    * **Output**: Corrected countrate (slope) image (`rate.fits`) 
>* **`STAGE 2`** ([calwebb_spec2](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2)): consists of additional instrument-level and observing mode corrections and calibrations.
    * **Input**: A single corrected countrate (slope) image (`rate.fits`) or an ASN file listing multiple inputs.
    * **Output**: A fully calibrated unrectified exposure (`cal.fits`). For NIRSpec IFU data, the `cube_build` step returns a 3-D IFU spectroscopic cube (`s3d.fits`). The `extract_1d` step  returns 1-D extracted spectral data products (`x1d.fits`)
>* **`STAGE 3`** ([calwebb_spec3](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html#calwebb-spec3)): consists of additional corrections (e.g. `outlier_detection`) and routines for combining calibrated data from multiple exposures (e.g. dither/nod pattern) into a single combined 2-D or 3-D spectral product and a combined 1-D spectrum. 
    * **Input**: An ASN file that lists multiple calibrated exposures (`cal.fits`).
    * **Output**: For NIRSpec IFU data, a resampled and combined 3-D IFU cube (`s3d.fits`) and a 1-D extracted spectrum (`x1d.fits`)

Here, we will focus on the mechanics of processing "real" example data ([Tarantula Nebula](#Tarantula)) from Early Release Observations (ERO) Proposal ID 2729, including how to use associations for multi-exposure combination and how to interact and work with data models for each product. Our objective is to examine the automated products found in MAST and compare them to products generated with the most up-to-date version of the JWST calibration pipeline.

Most processing runs shown here use the default reference files from the Calibration Reference Data System (CRDS). Please note that pipeline software development is a continuous process, so results in some cases may be slightly different if using a subsequent version. There are also a few known issues with some of the pipeline steps in this build that we expect to be fixed in the near future. Until then, at various steps, we provide users with the current processing recommendations when running the pipeline manually.

## 2. Import Library <a id='imports'></a>
<hr style="border:1px solid gray">

In [ ]:
#Import Library

#--------------------------------------JWST Calibration Pipeline Imports-------------------------------------------

import jwst
import crds
from jwst import datamodels
from jwst.pipeline import Detector1Pipeline   #calwebb_detector1
from jwst.pipeline import Spec2Pipeline       #calwebb_spec2
from jwst.pipeline import Spec3Pipeline       #calwebb_spec3
from jwst.extract_1d import Extract1dStep     #Extract1D Individual Step

print("===============================================================================")
print("These are the installed pipeline and current operational CRDS context versions:")
print(" ")
print("JWST Calibration Pipeline Version={}".format(jwst.__version__))
print("Current Operational CRDS Context = {}".format(crds.get_default_context()))
print("===============================================================================")

#----------------------------------------------General Imports-----------------------------------------------------

import numpy as np
import warnings
warnings.filterwarnings('ignore') #Set to 'default' to turn warnings back on

#--------------------------------------------File Operation Imports------------------------------------------------

import glob
import os
import asdf
import json
import shutil
from IPython.display import JSON

#--------------------------------------------Astropy/Astroquery Imports--------------------------------------------

from astropy.io import fits
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch, SqrtStretch
from astropy.stats import sigma_clipped_stats
import astroquery
from astroquery.mast import Mast
from astroquery.mast import Observations

#------------------------------------------------Plotting Imports--------------------------------------------------

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Circle
import matplotlib.gridspec as grd
from matplotlib import cm

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline
# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
plt.rcParams.update({'font.size': 18})

## 3. Convenience Functions <a id='func'></a>
<hr style="border:1px solid gray">

A few function that will be used throughout the notebook

In [ ]:
def show_image(data_2d, vmin, vmax, xsize=15, ysize=15, title=None, zoom_in=None, aspect=1, scale='log', units='DN/s', cmap='jet'):
    """
    Function to generate a 2-D, log-scaled image of the data
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        2-D image to be displayed
    vmin : float
        Minimum signal value to use for scaling
    vmax : float
        Maximum signal value to use for scaling
    xsize, ysize: int
        Figure Size
    title : str
        String to use for the plot title
    zoom_in: list 
        Zoomed in Region of interest [xstart,xstop,ystart,ystop]
    aspect: int
        Aspect ratio of the axes
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear' or 'Asinh'
    units : str
        Units of the data. Used for the annotation in the color bar. Defualt is DN/s for countrate images
    cmap: str
        Color Map for plot
    """
    #-----------------------------------------Scaling Information----------------------------------------
    
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    elif scale == 'Asinh':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=AsinhStretch())
    
    #--------------------------------------------Set Up Figure-------------------------------------------

    fig = plt.figure(figsize=(xsize, ysize))
    ax = fig.add_subplot(1, 1, 1)
    
    im = ax.imshow(data_2d, origin='lower', norm=norm, aspect=aspect, cmap=cmap)

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    #Zoom in on a portion of the image? 
    if zoom_in:
        #inset axis 
        axins = ax.inset_axes([0.5, 0.6, 0.5, 0.3])
        
        axins.imshow(data_2d, origin="lower", norm=norm, aspect=aspect, cmap=cmap)
        
        # subregion of the original image
        axins.set_xlim(zoom_in[0], zoom_in[1])
        axins.set_ylim(zoom_in[2], zoom_in[3])
        axins.set_xticklabels([])
        axins.set_yticklabels([])
        ax.indicate_inset_zoom(axins, color="black",edgecolor="black", linewidth=3)

In [ ]:
def show_ifu_cubeslices(s3d_file_list, wavelength_slices=[], spaxel_locs=[], y_scale=None, cmap='jet', vmin_vmax = [[[0,15e1]]], save_figure=False, title=None, title_font = 30):
    """
    Function to that takes a 3-D IFU data cube and generates: 
    
    > 2-D cube slices based on wavelength (microns)
    > Associated 1-D spectrum for a designated spaxel (spatial pixel) in the data cube
    > Corresponding 3-D weight image giving the relative weights of the output spaxels
    
    Note: This function can accomidate multiple detectors plotted side-by-side. 
    The general format would follow [[detector 1 info], [detector 2 info]].

    Parameters
    ----------
    s3d_file_list: list of str
        3-D IFU data cube fits file list 
    wavelength_slices: tuple
        List of wavelength values (microns) at which to create 2-D slices. 
    spaxel_locs: tuple
        List of spaxel locations in which to plot the associated 1-D spectrum. (One spaxel location per slice)
    y_scale: tuple
        Y-axis limits for the associated 1-D spectrum of the spaxel. Default is to use the ymin and ymax of the data. 
    cmap: str
        Color Map 
    vmin_vmax: tuple
        Minimum & Maximum signal value to use for scaling (e.g., [[[vmin,vmax],[vmin,vmax]], [[vmin,vmax], [vmin,vmax]]])
    title: str
        Figure Title. Default is None. 
    title_font:int
        Title Font Size
    save_figure: bool
        Save figure?         
    """
    
    #---------------------------------------------- Set-up Figure -------------------------------------------------

    #Plot Slices From the Cube
    fig = plt.figure(figsize=(8*np.array(wavelength_slices).size,18))
    gs = grd.GridSpec(3, np.array(wavelength_slices).size, height_ratios=[1]*3, width_ratios=[1]*np.array(wavelength_slices).size, hspace=0.4,wspace=0.7)

    total_num_plots=3*np.array(wavelength_slices).size
    
    plot_count = 0
    #---------------------------------------------Open Files------------------------------------------------------
    
    for s3d_file in s3d_file_list:
        
        root=s3d_file[:-9] #Root file name 
        print(s3d_file)
        s3d = fits.open(s3d_file) #3-D IFU data cube fits file 
        x1d3 = datamodels.open(root+'_x1d.fits') #1-D Extracted Spectrum            
    
        #--------------------------------Wavelength & Surface Brightness/Flux Arrays------------------------------
    
        x1d3wave = x1d3.spec[0].spec_table.WAVELENGTH
            
        #--------------------------------------Data & Header Information------------------------------------------

    
        #SCI Extension: [Type:ImageHDU  Cards:92   Dimensions:(57, 61, 973)   Format:float32]
        cube = s3d[1].data #Science data
        wcs = WCS(s3d[1].header) #World Coordinate System (WCS) Transformation keywords 
        wmap = s3d[4].data #3-D weight image giving the relative weights of the output spaxels.
        cdelt1 = s3d[1].header['CDELT1']*3600. #Axis 1 coordinate increment at reference point 
        cdelt2 = s3d[1].header['CDELT2']*3600. #Axis 2 coordinate increment at reference point 
        cdelt3 = s3d[1].header['CDELT3'] #Axis 3 coordinate increment at reference point 
        crval3 = s3d[1].header['CRVAL3'] #third axis value at the reference pixel  

        #Wavelength range of the grating/filter combination
        wavstart = s3d[1].header['WAVSTART']
        wavend = s3d[1].header['WAVEND']
        s3d.close()
    
        #---------------------------------------------------Plots-------------------------------------------------
        
        cmap_custom = cm.colors.LinearSegmentedColormap.from_list("", ["darkred","darkturquoise","blue"])
        colors = cmap_custom(np.linspace(0, 1, np.array(wavelength_slices).size))

        #To Account for if NRS1 & NRS2 are both being plotted Side-by-side
        if len(wavelength_slices) != 1:
            if 'nrs1' in s3d_file:
                wavelengths = wavelength_slices[0]
                spaxel_loc = spaxel_locs[0]
                vmin_vmax_vals = vmin_vmax[0]
                
                if y_scale:
                    y_scales = y_scale[0]

            elif 'nrs2' in s3d_file:
                wavelengths = wavelength_slices[1]
                spaxel_loc = spaxel_locs[1]
                vmin_vmax_vals = vmin_vmax[1]
                if y_scale:
                    y_scales = y_scale[1]

        else:
            wavelengths = wavelength_slices[0]
            spaxel_loc = spaxel_locs[0]
            vmin_vmax_vals = vmin_vmax[0]
            if y_scale:
                    y_scales = y_scale[0]

            
        #Loop through each wavelength slices
        for i, wave_slice in enumerate(wavelengths):

            if float(wavstart)<=wave_slice*10**-6<=float(wavend):
                
                #--------------------------------------------2-D Cube Slice------------------------------------------------
            
                #Min & Max Image Values & Scaling
                if len(vmin_vmax_vals) != 1:
                    vmax_val = vmin_vmax_vals[i][1]
                    vmin_val = vmin_vmax_vals[i][0]
                else:
                    vmax_val = vmin_vmax_vals[0][1]
                    vmin_val = vmin_vmax_vals[0][0]

                slicewave = wave_slice
                nslice = int((slicewave - crval3)/cdelt3) #the slice of the cube we want to plot
                ax1 = plt.subplot(gs[0+plot_count], projection=wcs, slices=('x', 'y', nslice)) #set up the subplot space
                #ax1 = plt.subplot(3,len(wavelength_slices), 0+plot_count, projection=wcs, slices=('x', 'y', nslice)) #set up the subplot space

                slice_mean = np.nanmean(cube[(nslice-2):(nslice+2), :, :], axis=0) #Mean of the slice looking in the range (nslice-2):(nslice+2)
                slice_norm=ImageNormalize(slice_mean, vmin=vmin_val, vmax=vmax_val, stretch=AsinhStretch()) #normalize &stretch 
                slice_image= ax1.imshow(slice_mean, norm=slice_norm, origin='lower', aspect='auto',cmap=cmap) #plot slice

                cb_image = fig.colorbar(slice_image, fraction=0.046, pad=0.04)
                cb_image.set_label('MJy/sr', labelpad=-1, fontsize = 22)
                cb_image.ax.tick_params(labelsize=20)
                cb_image.ax.yaxis.get_offset_text().set_fontsize(20)
                
                ax1.set_xlabel('RA', fontsize =22)
                ax1.set_ylabel('DEC', labelpad=-1, fontsize=22)
                #ax1.grid(color='white', ls='solid')
                ax1.set_title('Detector {} \n Grating/Filter: {}/{} \n {} microns'.format(s3d[0].header['DETECTOR'],s3d[0].header['GRATING'], s3d[0].header['FILTER'], str(slicewave)), fontsize =25)
                ax1.tick_params(axis='both', which='major', labelsize=20)
                ax1.coords[0].set_ticklabel(rotation=13, ha='right', pad=24)

               
                #------------------------------------------Spaxel 1-D Spectrum---------------------------------------------
                
                #Zoom in on a Spaxel: Spectrum
                loc = [spaxel_loc[i][0],spaxel_loc[i][1]]
                x1d3flux_loc = cube[:, loc[1], loc[0]]
                #ax2 = plt.subplot(3,len(wavelength_slices), int(total_num_plots/3)+plot_count)
                ax2 = plt.subplot(gs[int(total_num_plots/3)+plot_count])

                #Spaxel Box Highlight 
                spaxel_rect = plt.Rectangle((loc[0]-.5, loc[1]-.5), 1,1, fill=False, color='black', linewidth=2)
                ax1.add_patch(spaxel_rect)
                
                ax2.plot(x1d3wave, x1d3flux_loc, linewidth=1, color=colors[i])
                ax2.grid(linewidth=2)
                ax2.set_xlabel('$\u03BB [\u03BC$m]',fontsize=22)
                ax2.set_ylabel("Surface Brightness \n (MJy/sr)",fontsize=22)
                ax2.set_title('Spaxel at (x, y)='+repr(loc), fontsize=25)
                ax2.tick_params(axis='both', which='major', labelsize=20)
                ax2.yaxis.get_offset_text().set_fontsize(15)
                
                #Scale Information
                if y_scale:
                    ymin, ymax = y_scales[i][0], y_scales[i][1]
                else:
                    ymin, ymax = ax2.set_ylim()
                
                ax2.set_ylim(ymin, ymax)
                ax2.xaxis.set_tick_params(labelsize=20)
                ax2.yaxis.set_tick_params(labelsize=20)
                ax2.set_aspect(0.5/ax2.get_data_ratio())
                
                #-----------------------------------------------Weight Map-------------------------------------------------
                
                #Corresponding Weight Map (wmap) for Cube Slice
                ax3 = plt.subplot(gs[int(total_num_plots)-np.array(wavelength_slices).size+plot_count], projection=wcs, slices=('x', 'y', nslice)) #set up the subplot space
                #ax3 = plt.subplot(3, len(wavelength_slices), int(total_num_plots)-len(wavelength_slices)+plot_count, projection=wcs, slices=('x', 'y', nslice)) #set up the subplot space
                
                slice_mean_wmap = np.nanmean(wmap[(nslice-2):(nslice+2), :, :], axis=0) #Mean of the wmap slice looking in the range (nslice-2):(nslice+2)
                slice_norm_wmap=ImageNormalize(slice_mean_wmap, stretch=AsinhStretch()) #normalize &stretch
                slice_wmap = ax3.imshow(slice_mean_wmap, norm=slice_norm_wmap, origin='lower',aspect='auto', cmap=cmap) #plot slice

                cb_wmap = fig.colorbar(slice_wmap, fraction=0.046, pad=0.04)
                cb_wmap.set_label('Weight', labelpad=-1, fontsize = 22)
                cb_wmap.ax.tick_params(labelsize=20)
                cb_wmap.ax.yaxis.get_offset_text().set_fontsize(20)
                
                ax3.set_xlabel('RA', fontsize=22)
                ax3.set_ylabel('DEC', labelpad=-1, fontsize=22)
                #ax3.grid(color='gray', ls='solid')
                ax3.set_title(str(slicewave)+' microns: Weight Map', fontsize=25)
                ax3.tick_params(axis='both', which='major', labelsize=20)
                ax3.coords[0].set_ticklabel(rotation=13, ha='right', pad=24)

                plot_count += 1
                    
            else:
                None
    
    if title:
        fig.suptitle(title, fontsize=title_font)
        plt.subplots_adjust(top=0.8) 
    
    # fig.tight_layout(rect=[0, 0, 0.98, 0.98])

    if save_figure == True:
        fig.savefig(root+".png",dpi=24, bbox_inches="tight")
    

## 4. Directory Set-Up and defining some base parameters <a id='dir_setup'></a>
<hr style="border:1px solid gray">

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> Note: For this JWebbinar we pre-computed **all but one** products for [calwebb_detector1](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1) and [calwebb_spec2](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipelin/calwebb_spec2.html#calwebb-spec2) to save time. Those will be copied into the output folder. If `jwebbinar = False` **ALL** products are created, which can take a significant amount of time. For [calwebb_spec3](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html#calwebb-spec3) this is not possible since all products are needed to create the final datacube. Here we only reduce the G235H/F170LP grating. The G140H/F100LP and G395H/F290LP gratings are also pre-computed and compied into the output folder for `jwebbinar = True`.
</div>

In [ ]:
jwebbinar = True

if not jwebbinar:
    print('')
    print('JWebbinar is deactivated: ALL OBSERATIONS WILL BE REDUCED!!!')

hub_root = "/home/shared/preloaded-fits/jwebbinar-28"
user_root = "/home/jovyan/jwebbinar_prep/jwebbinar28"

# basedir = os.path.join(os.getcwd(),'nirspec_ifu_extended_source')
basedir_in = os.path.join(hub_root,'nirspec_ifu_extended_source')
basedir_out = os.path.join(user_root,'nirspec_ifu_extended_jwebbinar28')
output_dir = os.path.join(basedir_out,'run_folder')
mast_products_dir = os.path.join(basedir_in,'mast_products')
pre_computed_run_dir = os.path.join(basedir_in,'pre_computed_run')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.path.exists(mast_products_dir):
    os.makedirs(mast_products_dir)

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> Note: To allow everyone to get the same results we fixed the used PMAP to `jwst_1146.pmap`. To save download time we also provide the CRDS cache and to not overwrite any of your systems settings we saved the cache in  "../crds_cache". The following commands
`os.environ['CRDS_PATH'] = "./crds_cache"`, and
`os.environ['CRDS_CONTEXT'] = 'jwst_1146.pmap'`
overwrite the system settings for **THIS NOTEBOOK ONLY**. To run the notebook with the latest reference files these two lines should be commented out
</div>

In [ ]:
print('')
print("Setting up the CRDS-cache and pmap")

'''
The following should be uncommented to used the already cached files to save 
download time. Set if different to default set in bash_rc file. The CRDS_CONTEXT
ensures that everybody uses the same pmap for this exercise.
This MUST commented out to ensure that the latest reference files in CRDS
will be used
'''

os.environ['CRDS_PATH'] = "/home/jovyan/crds_cache" 
os.environ['CRDS_CONTEXT'] = 'jwst_1146.pmap' ### 

print('CRDS-cache is set to:', os.getenv('CRDS_PATH'))
print("Current activated CRDS Context = {}".format(os.getenv('CRDS_CONTEXT')))

## 5. Download the Data <a id='data'></a>

<hr style="border:1px solid gray">

| Target: Tarantula Nebula |       |   |   |   |
|:-----------:|:-------:|:---:|---|---|
| Proposal ID | 02729 |   |   |   |
| [GRATING/FILTER](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-observing-modes/nirspec-ifu-spectroscopy) | G140H/F100LP | λ: 0.97–1.89 μm (a medium resolution, R ~ 1000) |   |   |
|                | G235H/F170LP | λ: 1.66–3.17 μm (a high resolution, R ~ 2700) |   |   |
|                | G395H/F290LP | λ: 2.87–5.27 μm (a high resolution, R ~ 2700) |   |   |
|   DURATION  | 87.533 [s] | Total duration of one exposure |   |   |   |
|   READPATT  | NRSIRS2RAPID | Readout Pattern |   |   |   |
|   PATTTYPE  | CYCLING | Primary dither pattern type |   |   |
|   PATTSIZE  | LARGE | Primary dither pattern size (1.0" extent) |   |   |
|   NUMDTHPT  | 8 | Total number of points in pattern |   |   | 
|   SRCTYAPT  | UNKNOWN | Source Type selected in APT |   |   | 

> **Note:** The presence of a physical gap between detectors affects high-resolution IFU observations because the spectra are long enough to span both NIRSpec detectors. When using the grating-filter combination G140H/F070LP, PRISM/CLEAR, or the M-gratings, the resulting spectra do not have any gaps because the spectra do not extend beyond NRS1. [More Info ...](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-operations/nirspec-ifu-operations/nirspec-ifu-wavelength-ranges-and-gaps#NIRSpecIFUWavelengthRangesandGaps-Wavelengthgaps)

MAST products have already been pre-downloaded and stored in a provided demo directory.

## 6. Products Found In MAST <a id='mast_products'></a>
<hr style="border:1px solid gray">

Here we show and plot the products as they are provided in MAST. This will also demonstrate why in most science cases reprocessing the data is needed.

<div class="alert alert-block alert-info">

<b>Tip:</b> To optimally use your science data some parameters need to be uniquly tweaked specifically for your observations. No parameter setting wil be sufficient for **all** data.
    
</div> 

> In [APT](https://jwst-docs.stsci.edu/jwst-astronomers-proposal-tool-overview), the observer has three options for source type (`SRCTYAPT` keyword): `POINT`, `EXTENDED`, or `UNKNOWN`. In stage 2, the `srctype` step will first check if the `SRCTYAPT` keyword is present and populated. If `SRCTYAPT` is not present or is set to `UNKNOWN`, the step determines a suitable value based on the observing mode, command line input, and other characteristics of the exposure. If the exposure is identified as a background exposure (`BKGDTARG = True`), the exposures default to a source type of `EXTENDED`. Exposures that are part of a nodded pattern (identified by keyword `PATTYPE`), which are assumed to only be used with point-like targets, default to a source type of `POINT`. [More Info ...](https://jwst-pipeline.readthedocs.io/en/latest/jwst/srctype/description.html#single-source-observations)

For dithered NIRSpec IFU data like ours, which do not meet any of the above conditions, will default to source type `EXTENDED`. <mark> Therefore, the products found in MAST for target Tarantula Nebula (PID 2729) have been processed as an `EXTENDED` source </mark>. 

### 6.1 Stage 1 Products Found In MAST  <a id='level1_mast'></a>
<hr style="border:1px solid gray">

In [ ]:
#Stage 1 slope products -- level 2a images

#Plot 4th (out of 8) dither position (spectra fall on both NRS1 & NRS2) for GRATING/FILTER G235H/F170LP combination  

file = glob.glob(os.path.join(mast_products_dir, '*02103*00004_nrs1_rate.fits'))[0]
print('rate file: ',file)
ratefile_open = datamodels.open(file)
ratefile_sci = ratefile_open.data #get the pixel data (the SCI extension of the fits file)
ratefile_dq = ratefile_open.dq #data quality map data (DQ extension)
             
#Plot the slope image and small section of the countrate image & corresponding section of the DQ map
show_image(ratefile_sci, 0,10, units='DN/s',zoom_in=[650,700, 1250,1300], ysize=20, xsize=20,
           title='Countrate Image \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) #rate files have units of DN/s
 
show_image(ratefile_dq, 0, 10, units='Bit Value', scale='linear',zoom_in=[650,700, 1250,1300], ysize=20, xsize=20,
           title='Data Quality Map \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) 

file = glob.glob(os.path.join(mast_products_dir, '*02103*00004_nrs2_rate.fits'))[0]
print('rate file: ',file)
ratefile_open = datamodels.open(file)
ratefile_sci = ratefile_open.data #get the pixel data (the SCI extension of the fits file)
ratefile_dq = ratefile_open.dq #data quality map data (DQ extension)
             
#Plot the slope image and small section of the countrate image & corresponding section of the DQ map

show_image(ratefile_sci, 0,10, units='DN/s',zoom_in=[650,700, 1720,1770], ysize=20, xsize=20,
           title='Countrate Image \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) #rate files have units of DN/s
 
show_image(ratefile_dq, 0, 10, units='Bit Value', scale='linear',zoom_in=[650,700, 1720,1770], ysize=20, xsize=20,
           title='Data Quality Map \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) 

<div class="alert alert-block alert-danger">
    
<b>Warning:</b> Please be aware that in the countrate (slope) images found in MAST, many pixels are flagged as Do Not Use (more clearly seen in the corresponding DQ map) and therefore appear white with a value of NaN. This excessive flagging is due to an outdated mask reference file that would mark unreliable slope, bad fit, and telegraph pixels as Do Not Use. Despite the large number of NaNs in the countrate image, the extracted spectra are not significantly affected by them when combining multiple dithered exposures because the number of flagged pixels is still a relatively small fraction. However, due to the high number of flags in the MAST products, it is difficult to see specific details in the slope images, like correlated read noise, which manifests as low-level vertical banding/striping and a "picture frame" with the [$IRS^{2}$](https://jwst-docs.stsci.edu/jwst-near-infrared-spectrograph/nirspec-instrumentation/nirspec-detectors/nirspec-detector-readout-modes-and-patterns/nirspec-irs2-detector-readout-mode) readout mode. As of context jwst_1084.pmap, the pipeline now considers unreliable slope, bad fit, and telegraph pixels good for further processing in Full frame data. [Therefore, the reprocessed data below offers improved visibility of the correlated read noise.](#level1_rerun)

</div>

<div class="alert alert-block alert-danger">
    
<b>Warning:</b> Please be aware that many exposures are suffering from snowballs (see NRS2 in the above plot) which are caused by large cosmic ray events (https://jwst-docs.stsci.edu/data-artifacts-and-features/snowballs-and-shower-artifacts). To mitigate this ``expand_large_events`` needs to be set to `True` in the jump_detection algorithm of the `callwebb_detector1` step. This is currently deactivate by default hence a rerun is needed. This is especially important for deep exposures with long integration times. A rough estimate is 1 snowball per detector per 20s.

</div>

### 6.2 Stage 2 Products Found In MAST  <a id='level2_mast'></a>
<hr style="border:1px solid gray">

In [ ]:
#Stage 2 Products -- Calibrated 3-D data cubes for each GRATING/FILTER combination

#Plotting the 4th (out of 8) dither position for both NRS1 and NRS2
s3d_g140h_stage2 = sorted(glob.glob(os.path.join(mast_products_dir, '*2105*00004_nrs?_s3d.fits')))
s3d_g235h_stage2 = sorted(glob.glob(os.path.join(mast_products_dir, '*2103*00004_nrs?_s3d.fits')))
s3d_g395h_stage2 = sorted(glob.glob(os.path.join(mast_products_dir, '*2101*00004_nrs?_s3d.fits')))
s3d_stage2_list = s3d_g140h_stage2+s3d_g235h_stage2+s3d_g395h_stage2

title_stage2_mast='Tarantula Nebula \n Level 2 IFU Product: 3-D Cube Slices vs. Corresponding 3-D Weighted Map'

#Characteristics of the plot 
nrs1_wavelengths = [1.0,2.3,3.4] #Wavelength slices (microns) to take from the 3-D data cube
nrs2_wavelengths = [1.4,2.5,4.0]

nrs1_spaxel_locs = [[30,29],[28,39],[14,25]] #Spaxel locations for associated 1-D spectrum (one spaxel plotted per slice)
nrs2_spaxel_locs = [[30,29],[28,39],[14,25]]

nrs1_vmin_vmax = [[0,2e2],[0,1e2],[0,1.2e3]] #Minimum & Maximum signal values for scaling each slice
nrs2_vmin_vmax = [[0,2e2],[0,1e2],[0,1.2e3]]

nrs1_yscales = [[-80,150], [-80,150], [-80,200]] #Spaxel plot y-limits
nrs2_yscales = [[-80,200], [-80,150], [-80,200]]

#Plot using the convience function defined above
show_ifu_cubeslices(s3d_stage2_list, wavelength_slices=[nrs1_wavelengths,nrs2_wavelengths], 
                    spaxel_locs=[nrs1_spaxel_locs,nrs2_spaxel_locs],vmin_vmax=[nrs1_vmin_vmax,nrs2_vmin_vmax], 
                    y_scale = [nrs1_yscales,nrs2_yscales], title=title_stage2_mast)

### 6.3 Stage 3 Products Found In MAST  <a id='level3_mast'></a>
<hr style="border:1px solid gray">

In [ ]:
#Stage 3 Products -- Combined Calibrated 3-D data cubes for each GRATING/FILTER combination

s3d_stage3_list = sorted(glob.glob(os.path.join(mast_products_dir, '*nirspec*_s3d.fits')))

title_stage3_mast='Tarantula Nebula \n Level 3 IFU Product: 3-D Cube Slices vs. Corresponding 3-D Weighted Map'

#Characteristics of the plot 
wavelengths = [1.4,2.3,4.0] #Wavelength slices (microns) to take from the combined 3-D data cube
spaxel_locs = [[29,32],[29,32],[29,32]] #Spaxel locations for associated 1-D spectrum (one spaxel plotted per slice)
vmin_vmax = [[0,2e2],[0,1e2],[0,1.2e3]] #Minimum & Maximum signal values for scaling each slice
yscales = [[0,2e3], [0,7e3], [0,2e4]]

#Plot using the convience function defined above
show_ifu_cubeslices(s3d_stage3_list, wavelength_slices=[wavelengths],spaxel_locs=[spaxel_locs],
                    vmin_vmax=[vmin_vmax], y_scale = [yscales], title=title_stage3_mast)

<div class="alert alert-block alert-danger">
    
<b>Warning:</b> Please note that in the final product (stage 3) downloaded from MAST, a significant portion of the data got rejected, returning a value of zero in the weight maps. This over-rejection of data is due to the outdated `outlier_detection` step that MAST automatically enables during stage 3 of the pipeline. A new outlier detection algorithm has been developed specifically for IFU data that overcomes some of these limitations (as of DMS build B9.3rc1/CAL_VER 1.11.0). Due to the limitations of the previous outlier detection algorithm, the user recommendation is to skip the `outlier_detection` step if using an older version of the pipleine or manually rerun stage 3 of the pipleine with outlier detection on with the most up-to-date pipeline version (detailed in the next section of this notebook).
</div>

<div class="alert alert-block alert-info">
    
<b>Note:</b> 
When the source type is extended, the default extraction aperture for the `extract_1d` step covers the entire cube.
Since also pixels at the edges of the FoV are used where often only 1 or 2 dither positions are available, the 1D spectrum also contains contributions from regions where the outlier rejection cannot work properly due to the lack of data. Therefore, we are extracting an arbitrary box of 4x4 spaxels (y: 29 to 33 and x: 27 to 30). Typically users are not collapsing their cubes to create a single spectrum for an extended source when using IFU data.
</div>

In [ ]:
#Just a check to see what verison of the pipeline and what pmap was used
x1d3_mast = fits.open(glob.glob(os.path.join(mast_products_dir, '*_x1d.fits'))[0])

print("Products found in MAST used JWST calibration pipeline version: {} and {}".format(x1d3_mast[0].header['CAL_VER'],x1d3_mast[0].header['CRDS_CTX']))

In [ ]:
#Stage 3 Products -- Combined Extracted 1-D Spectrum 

fig = plt.figure(figsize=(15,9))
colors = ['darkred', 'darkturquoise', 'blue']

x1d3_mast_list = sorted(glob.glob(os.path.join(mast_products_dir, '*nirspec*_x1d.fits')))

s3d_mast_list = sorted(glob.glob(os.path.join(mast_products_dir, '*nirspec*_s3d.fits')))


for i, x1d3_file in enumerate(x1d3_mast_list):
    x1d3_file_open = datamodels.open(x1d3_file) 
    cube = fits.open(s3d_mast_list[i])['SCI'].data

    #Wavelength & Surface Brightness Arrays
    x1d3wave_mast = x1d3_file_open.spec[0].spec_table.WAVELENGTH
    x1d3flux_mast = x1d3_file_open.spec[0].spec_table.SURF_BRIGHT

    plt.plot(x1d3wave_mast,np.mean(cube[:, 29:34,27:31], axis=(1,2)),color=colors[i], linewidth =2, label='Grating/Filter: {}/{}'.format(x1d3_file_open.meta.instrument.grating,
                                                                                             x1d3_file_open.meta.instrument.filter))

plt.xlabel('$\lambda [\mu$m]', fontsize =15)
plt.ylabel('Surface Brightness (MJy/sr)', fontsize =15)
plt.title("Tarantula Nebula \n Level 3 IFU Product in MAST: Extracted 1-D Spectrum", fontsize =20)
plt.ylim(-1e1, 5e3)
plt.legend()

<div class="alert alert-block alert-danger">
    
<b>Warning:</b> Most of the large negative and positive flux spikes extending beyond the plot range are likely due to bad/hot pixels that are not flagged in the current DQ masks. The mask reference file gets directly pulled from CRDS. The products found in MAST use a specific CRDS context (.pmap) when processing data. However, the CRDS is constantly updating the operational .pmap.
</div>

<div class="alert alert-block alert-danger">
    
<b>Warning:</b> The systematically lower flux \& wavy continua in the red portions of the spectrum from each grating are artifacts due to correlated read noise. This is apparent in the full frame images, manifesting as vertical banding often accompanied by a "picture frame" effect, and is caused by low-level detector instabilities that even the $IRS^{2}$ algorithm cannot remove. The effect is typically only noticeable in read-noise limited data, and is most prominent on the NRS2 detector. The pipeline currently does not apply any correction for this, but there is an external algorithm, "NSClean", developed by Bernie Rauscher at GSFC, that is available to the public; see https://webb.nasa.gov/content/forScientists/publications.html#NSClean for details.

</div>

## 7. Re-processing the Data <a id='reprocessing'></a>
<hr style="border:1px solid gray">

<div class="alert alert-block alert-info">
    
<b>Note:</b> Many parameter we show in the below examples are set to the default ones. This should only provide an overview over some of the most common parameters that can be adjusted to get the best out of the science **but** often they require a few iterations to find the best possible parameter set for a given observation. 
</div>

Due to lengthy processing times, only one exposure of the observations (G235H/F170LP) will be re-processed in this demonstration if `jwebbinar=True`. 

### 7.1 Stage 1 Rerun & Products  <a id='level1_rerun'></a>
<hr style="border:1px solid gray">

<div class="alert alert-block alert-info">
    
<b>Note:</b> The most notebale difference to the MAST products will be the snowball rejection, which are large cosmic ray event. This can be acomplished setting `"expand_large_events": True` in the `jump` step. Setting the `"expand_factor": 3` seems to be a good value for NIRSpec observations to cover most snowballs. A manual check of the rate files is recommended.
</div>

In [ ]:
#Stage 1 Processing 

if jwebbinar == True:
    filelist = sorted(glob.glob(os.path.join(mast_products_dir, '*02103*00004_nrs?_uncal.fits')))
else:
    filelist = sorted(glob.glob(os.path.join(mast_products_dir, '*_uncal.fits')))

for uncal_file in filelist: 
    print("Applying Stage 1 Corrections & Calibrations to: "+ os.path.basename(uncal_file))

    result = Detector1Pipeline.call(uncal_file,
                                    save_results = True,
                                    output_dir = output_dir,
                                    steps = {"jump":{"skip": False,
                                                     "expand_large_events": True, ### this activates the snowball rejection
                                                     "maximum_cores": "all", ### you can limit the number of used CPUs here
                                                     "expand_factor": 3, ### the non default value 3 seems to be better for NIRSpec to mask the full region
                                                     # "rejection_threshold": 4.0,
                                                     # "four_group_rejection_threshold": 5.0,
                                                     # "flag_4_neighbors": True,
                                                     # "max_jump_to_flag_neighbors": 200,
                                                     # "min_jump_to_flag_neighbors": 10,
                                                     # "edge_size": 4,
                                                     # "sat_expand": 0.,
                                                     # "sat_required_snowball": False,
                                                     # "min_jump_area": 2.,
                                                     "save_results": True,
                                                     },
                                             "refpix": {"skip": False,
                                                        "odd_even_columns": True,
                                                        # "use_side_ref_pixels": True,
                                                        # "side_smoothing_lenght": 11,
                                                        # "side_gain": 1.0,
                                                        "ovr_corr_mitigation_ftr": 3.0
                                                       },
                                             "ramp_fit": {"skip": False,
                                                         "suppress_one_group": False,
                                                         "maximum_cores": 'all'
                                                         },
                                             "saturation": {"skip": False,
                                                           "n_pix_grow_sat": 1
                                                           },
                                             
                                             })


if jwebbinar == True:

    pre_computed_filelist = sorted(glob.glob(os.path.join(pre_computed_run_dir, '*_rate*.fits')))
    for file in pre_computed_filelist:
        output = os.path.join(output_dir, os.path.basename(file))
        print(file, ' => ', output)
        shutil.copy(file, output)

print("Hurray ...  this step has been completed")

In [ ]:
#Stage 1 slope products -- level 2a images

#Plot 1 dither position (spectra fall on both NRS1 & NRS2) for GRATING/FILTER G235H/F170LP combination  

file = glob.glob(os.path.join(output_dir, '*02103*00004_nrs1_rate.fits'))[0]
print("rate file: ", file)

ratefile_open = datamodels.open(file)
ratefile_sci = ratefile_open.data #get the pixel data (the SCI extension of the fits file)
ratefile_dq = ratefile_open.dq #data quality map data (DQ extension)
             
#Plot the slope image and small section of the countrate image & corresponding section of the DQ map

show_image(ratefile_sci, 0,10, units='DN/s',zoom_in=[650,700, 1250,1300], ysize=20, xsize=20,
           title='Countrate Image \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) #rate files have units of DN/s
 
show_image(ratefile_dq, 0, 10, units='Bit Value', scale='linear',zoom_in=[650,700, 1250,1300], ysize=20, xsize=20,
           title='Data Quality Map \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) 

file = glob.glob(os.path.join(output_dir, '*02103*00004_nrs2_rate.fits'))[0]
print("rate file: ", file)

ratefile_open = datamodels.open(file)
ratefile_sci = ratefile_open.data #get the pixel data (the SCI extension of the fits file)
ratefile_dq = ratefile_open.dq #data quality map data (DQ extension)

#Plot the slope image and small section of the countrate image & corresponding section of the DQ map
show_image(ratefile_sci, 0,10, units='DN/s',zoom_in=[650,700, 1720,1770], ysize=20, xsize=20,
           title='Countrate Image \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter)) #rate files have units of DN/s
 
show_image(ratefile_dq, 0, 10, units='Bit Value', scale='linear',zoom_in=[650,700, 1720,1770], ysize=20, xsize=20,
           title='Data Quality Map \n Detector: {} \n 8-Cycle Dither Position Index: {} \n GRATING/FILTER: {}/{}'.format(ratefile_open.meta.instrument.detector,
                                                                                                                        ratefile_open.meta.dither.position_number, 
                                                                                                                        ratefile_open.meta.instrument.grating,
                                                                                                                        ratefile_open.meta.instrument.filter))

<div class="alert alert-block alert-info">
    
<b>Note:</b> Compared to the [countrate (slope) products found in MAST](#level1_mast), fewer pixels are flagged as Do Not Use when using the most up-to-date pmap in CRDS. With the latest pmap, one can observe low-level vertical banding in the central regions of the detector, and the "picture frame" towards the edge of both detectors, where there is less correlated read noise a lot easier. 
</div>

### 7.2 Stage 2 Rerun & Products  <a id='level2_rerun'></a>
<hr style="border:1px solid gray">


During stage 2 of the pipeline, the countrate (slope) image products from stage 1, which have units of DN/s, are converted to units of surface brightness (MJy/sr) for both extended and point sources (as of DMS build 9.3/CAL_VER 1.10.2). For extended targets, like the Tarantula Nebula, the `extract_1d` step is controlled by a different set of parameters in the EXTRACT1D reference file: 

> For an extended source, rectangular aperture photometry is used, with the entire image being extracted, and no background subtraction, regardless of what was specified in the reference file or step arguments. [More Info ...](https://jwst-pipeline.readthedocs.io/en/latest/jwst/extract_1d/description.html)

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> Note there has been a bug in the `cube_build` step that caused the point source flux to not be conserved when using different spatial sampling. A fix has been implemented as of release DMS build 9.3/CAL_VER 1.10.2. In order to enable the correct functionality, the units of the cal.fits files and cubes will now be in surface brightness, and only the 1-D extracted spectra will be in units of Jy.
</div>

In [ ]:
#Stage 2 Processing 

if jwebbinar == True:
    filelist = sorted(glob.glob(os.path.join(output_dir, '*02103*00004_nrs?*rate.fits')))
else:
    filelist = sorted(glob.glob(os.path.join(output_dir, '*_rate.fits')))

#Process each rate file seperately 
for rate_file in filelist:
        
    print("Applying Stage 2 Calibrations & Corrections to: "+ os.path.basename(rate_file))
    
    result = Spec2Pipeline.call(rate_file,
                                save_results = True,
                                output_dir = output_dir,
                                steps = {"msa_flagging":{"skip": False   ### Masks those pixels that are affected by stuck open MSA shutters
                                                         },
                                         "imprint_subtract":{"skip": True    ### This step is needed if LEAKCAL observations were provided"
                                                    },
                                         "bkg_subtract":{"skip": True,    ### This step is needed if background observations were provided
                                                       "sigma": 3,
                                                       "maxiters": None,
                                                       "save_combined_background": False
                                                       },
                                          "flat_field":{"skip": False,
                                                        "save_interpolated_flat": False   ### A flag to indicate whether to save to a file the NIRSpec flat field that was constructed on-the-fly by the step.
                                                        },
                                         "pathloss":{"skip": False
                                                     },
                                         "photom":{"skip": False   ### photmetric calibration 
                                                   },
                                         "cube_build":{"skip": False    ### builds the 3D cube for each exposure. This is not necessary in the Spec2 step unless you want to inspect the individual cubes before combining them
                                                       },
                                         "extract_1d":{"skip": False   ### Extracts the 1D spectra for each exposure . Is not necessary in the Spec2 step unless you want to inspect the individual extracted spectra
                                                       }
                                         }
                                )

if jwebbinar == True:

    pre_computed_filelist = sorted(glob.glob(os.path.join(pre_computed_run_dir, '*_cal*.fits')))
    for file in pre_computed_filelist:
        output = os.path.join(output_dir, os.path.basename(file))
        print(file, ' => ', output)
        shutil.copy(file, output)

print("Hurray ...  this step has been completed")

In [ ]:
#Stage 2 Products -- Calibrated 3-D data cubes for each GRATING/FILTER combination

#Plotting the 4th (out of 8) dither position for both NRS1 and NRS2
s3d_stage2_list = sorted(glob.glob(os.path.join(output_dir, '*2103*00004_nrs?_s3d.fits')))

title_stage2_rerun='Tarantula Nebula \n Level 2 IFU Product: 3-D Cube Slices vs. Corresponding 3-D Weighted Map'

#Characteristics of the plot 
nrs1_wavelengths = [2.3] #Wavelength slices (microns) to take from the 3-D data cube
nrs2_wavelengths = [2.5]

nrs1_spaxel_locs = [[28,39]] #Spaxel locations for associated 1-D spectrum (one spaxel plotted per slice)
nrs2_spaxel_locs = [[28,39]]

nrs1_vmin_vmax = [[0,1e2]] #Minimum & Maximum signal values for scaling each slice
nrs2_vmin_vmax = [[0,1e2]]

nrs1_yscales = [[-80,150]] #Spaxel plot y-limits
nrs2_yscales = [[-80,150]]

#Plot using the convience function defined above
show_ifu_cubeslices(s3d_stage2_list, wavelength_slices=[nrs1_wavelengths,nrs2_wavelengths], 
                    spaxel_locs=[nrs1_spaxel_locs,nrs2_spaxel_locs],vmin_vmax=[nrs1_vmin_vmax,nrs2_vmin_vmax], 
                    y_scale = [nrs1_yscales,nrs2_yscales], title=title_stage2_rerun)

### 7.3 Stage 3 Rerun & Products  <a id='level3_rerun'></a>
<hr style="border:1px solid gray">

***Level 3 ASN File***

> Observations that use a nod-type/dither patterns, their exposures are related. [Association files (ASN)](https://jwst-pipeline.readthedocs.io/en/stable/jwst/associations/overview.html) describe how multiple exposures are related to one another and how they depend on one another. Processing an ASN file permits exposures to be calibrated, archived, retrieved, and reprocessed as a set rather than individual objects. IFU exposures taken with a dither pattern are not used for pixel-to-pixel background subtraction by the calibration pipeline (unlike exposures taken with a nod pattern).

Therefore, all calibration files (`cal.fits`) in our spec3 ASN file should be labeled as science exposures (`exptype: science`).  

In [ ]:
#Copy ASN file from MAST

asnfiles_mast = glob.glob(os.path.join(mast_products_dir, '*_spec3_*_asn.json')) #ASN file found in MAST

for file in asnfiles_mast:

    asnfile_dest = os.path.join(output_dir, os.path.basename(file))
    
    print(file, " => " , asnfile_dest)
    shutil.copy(file, asnfile_dest)

In [ ]:
### Show the G235H/F170LP ASN file for which we will run CalSpec3

asnfile_g140h = glob.glob(os.path.join(output_dir, '*_spec3_00003_asn.json'))[0]
asnfile_g235h = glob.glob(os.path.join(output_dir, '*_spec3_00001_asn.json'))[0]
asnfile_g395h = glob.glob(os.path.join(output_dir, '*_spec3_00002_asn.json'))[0]

with open(asnfile_g235h, 'r') as f_obj:
    asnfile_data = json.load(f_obj)

JSON(asnfile_data, expanded=True)

#### 7.3.1 New Outlier Detection Algorithm<a id='outlier_detection_new'></a>
<hr style="border:1px solid gray">

The new outlier detection algorithm for IFU data (as of DMS build B9.3rc1/CAL_VER 1.11.0) implements the basic outlier detection algorithm -- searches for pixels that are consistent outliers in the calibrated images created by the `calwebb_spec2` pipeline. The algorithm generally operates as follows:

> * Identifies outlier pixels by comparing them with their neighboring pixels in the spatial direction across a set of input files within an association.
> * For NIRSpec data, it calculates differences between pixels located above and below each science pixel.
> * The pixel differences for every input model in the association are computed and stored in a stack of pixel differences.
> * For each pixel, the algorithm determines the minimum difference across this stack and then performs normalization. This normalization process employs a local median derived from the difference array, with the size of the median determined by the kernel size.
> * A pixel is flagged as an outlier if this normalized minimum difference is greater than the input threshold percentage. 
> * Pixels that are found to be outliers are flaged in in the DQ array.
> * [More Info ...](https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/outlier_detection_ifu.html#outlier-detection-ifu)


**[The outlier_detection step for IFU data has the following optional arguments that control the behavior of the processing](https://github.com/spacetelescope/jwst/blob/master/docs/jwst/outlier_detection/arguments.rst):**

* `kernel_size` (string, default='7 7'): The size of the kernel to use to normalize the pixel differences. The kernel size must only contain odd values.
* `threshold_percent` (float, default=99.8): The threshold (in percent) of the normalized minimum pixel difference used to identify bad pixels. Pixels with a normalized minimum pixel difference above this percentage are flagged as a outlier.
* `save_intermediate_results` (boolean, default=False): Specifies whether or not to save any intermediate products created during step processing.

<div class="alert alert-block alert-info">
    
<b>Note:</b> The default `kernel_size` of **7 7** was developed for MIRI/MRS and tests showed that **3 3** works the better option for NIRSpec.
</div>


In [ ]:
#Rerun stage 3

if jwebbinar == True:
    filelist = [asnfile_g235h]
else:
    filelist = [asnfile_g140h, asnfile_g235h, asnfile_g395h]

for asn_file in filelist:

    result = Spec3Pipeline.call(asn_file,
                                save_results = True,
                                output_dir = output_dir,
                                steps = {"outlier_detection":{"skip": False,
                                                              "save_results": True,
                                                              "kernel_size": '3 3',    # the default of 7 7 was developed for MIRI/MRS and from testing 3 3 is the better option for NIRSpec
                                                              "threshold_percent": 99.8
                                                             },
                                        "cube_build":{"skip": False,
                                                      # "gratings": "ALL",      ### “ALL” is used, then all the gratings in the association are used.
                                                      # "output_type": "multi", ### combines data into a single “uber” IFU cube "
                                                      "weighting": 'emsm'    ### From testing emsm seems to be working better than drizzle for NIRSpec
                                                     },
                                        "extract_1d":{"skip": False
                                                      # "center_xy" = "27, 28"  ### A list of two integer values giving the desired x/y location for the center of the circular extraction aperture,
                                                      # "ifu_autocen" = True  ### Switch to select whether or not to enable auto-centroiding of the extraction aperture for IFU point sources
                                                     }
                                        })


if jwebbinar == True:

    pre_computed_filelist = sorted(glob.glob(os.path.join(pre_computed_run_dir, '*lp_??d.fits')))
    for file in pre_computed_filelist:
        output = os.path.join(output_dir, os.path.basename(file))
        print(file, ' => ', output)
        shutil.copy(file, output)

print("Hurray ...  this step has been completed")

In [ ]:
#Stage 3 Products -- Combined Calibrated 3-D data cubes for GRATING/FILTER combination G235H/F170LP

s3d_stage3_list = sorted(glob.glob(os.path.join(output_dir, '*nirspec*_s3d.fits')))

title_stage3_mast='Tarantula Nebula \n Level 3 IFU Product: 3-D Cube Slices vs. Corresponding 3-D Weighted Map'

#Characteristics of the plot 
wavelengths = [1.4,2.3,4.0] #Wavelength slices (microns) to take from the combined 3-D data cube
spaxel_locs = [[29,32],[29,32],[29,32]] #Spaxel locations for associated 1-D spectrum (one spaxel plotted per slice)
vmin_vmax = [[0,2e2],[0,1e2],[0,1.2e3]] #Minimum & Maximum signal values for scaling each slice
yscales = [[0,2e3], [0,7e3], [0,2e4]] #Spaxel plot y-limits

#Plot using the convience function defined above
show_ifu_cubeslices(s3d_stage3_list, wavelength_slices=[wavelengths],spaxel_locs=[spaxel_locs],
                    vmin_vmax=[vmin_vmax], y_scale = [yscales], title=title_stage3_mast)

<div class="alert alert-block alert-info">
    
<b>Note:</b> In comparison to the [weight maps for the 3-D data cube products found in MAST](#level3_mast), the implementation of the new outlier detection algorithm leads to a notable decrease in data rejection.

<b>Note:</b> 
When the source type is extended, the default extraction aperture for the `extract_1d` step covers the entire cube.
Since also pixels at the edges of the FoV are used where often only 1 or 2 dither positions are available, the 1D spectrum also contains contributions from regions where the outlier rejection cannot work properly due to the lack of data. Therefore, we are extracting an arbitrary box of 4x4 spaxels (y: 29 to 33 and x: 27 to 30). Typically users are not collapsing their cubes to create a single spectrum for an extended source when using IFU data.
</div>

In [ ]:
#Stage 3 Products -- Combined Extracted 1-D Spectrum 

fig = plt.figure(figsize=(15,9))
colors = ['darkred', 'darkturquoise', 'blue']

x1d3_rerun_list = sorted(glob.glob(os.path.join(output_dir, '*nirspec*_x1d.fits')))
s3d_rerun_list = sorted(glob.glob(os.path.join(output_dir, '*nirspec*_s3d.fits')))

x1d3_mast_list = sorted(glob.glob(os.path.join(mast_products_dir, '*nirspec*_x1d.fits')))
s3d_mast_list = sorted(glob.glob(os.path.join(mast_products_dir, '*nirspec*_s3d.fits')))

labels = ['', '', 'MAST product']

for i, x1d3_file in enumerate(x1d3_rerun_list):
    x1d3_file_open = datamodels.open(x1d3_file) 
    
    #Wavelength & Surface Brightness Arrays
    x1d3wave_rerun = x1d3_file_open.spec[0].spec_table.WAVELENGTH
    x1d3flux_rerun = x1d3_file_open.spec[0].spec_table.SURF_BRIGHT

    cube = fits.open(s3d_rerun_list[i])['SCI'].data
    plt.plot(x1d3wave_rerun,np.mean(cube[:, 29:34,27:31], axis=(1,2)),color=colors[i], linewidth =2, label='Grating/Filter: {}/{}'.format(x1d3_file_open.meta.instrument.grating,
                                                                                             x1d3_file_open.meta.instrument.filter), zorder=2)
    x1d3_mast_file_open = datamodels.open(x1d3_mast_list[i]) 
    x1d3wave_mast = x1d3_mast_file_open.spec[0].spec_table.WAVELENGTH

    mast_cube = fits.open(s3d_mast_list[i])['SCI'].data
    plt.plot(x1d3wave_mast,np.mean(mast_cube[:, 29:34,27:31], axis=(1,2)),color='grey',alpha=0.5, linewidth =2, zorder=0, label = labels[i])
    

#Where wavelength slice was taken above
plt.xlabel('$\lambda [\mu$m]', fontsize =15)
plt.ylabel('Surface Brightness (MJy/sr)', fontsize =15)
plt.title("Tarantula Nebula \n Level 3 IFU Product in MAST: Extracted 1-D Spectrum", fontsize =20)
plt.ylim(-1e1, 6e3)
plt.legend()

## Conclusion <a id='conclusion'></a>
<hr style="border:1px solid gray">

In conclusion, this notebook walks users through processing real data (Tarantula Nebula) from ERO Proposal ID 2729 and comparing automated products in MAST with those generated using the latest version of the JWST calibration pipeline and latest CRDS context. For optimal results, users are strongly encouraged to reprocess their own data using the most recent pipeline version and CRDS context, taking advantage of bug fixes and algorithm improvements (i.e., the new IFU outlier detection algorithm). 